<a href="https://colab.research.google.com/github/joaopedsa/TCC/blob/master/TCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Execute esta célula para preparar a interface visual

#Importacao via python da biblioteca fastai
![ -e /content ] | tee -a >> log.txt && pip install ipywidgets | tee -a >> log.txt && jupyter nbextension enable --py widgetsnbextension | tee -a >> log.txt

#Path Lib
import os
from pathlib import Path
import glob
import re

#Path.ls
from fastai import core

#Importações do google colab
from google.colab import drive
from google.colab import widgets as colabwidigets

import matplotlib.pyplot as plt
import numpy as np

#Pytorch
import torch

#Images
from IPython.display import Image, display, clear_output

#Importações Ipywidigets
from ipywidgets import widgets as ipywidgets

class m_Header():
  def __init__(self, header):
        self.header = ipywidgets.HTML(value='<p style=font-family:Arial><b>' + header + '<b/> <a target="_blank" href="https://github.com/joaopedsa/TCC/blob/master/README.md" > ?</a><p>')

class m_Ret():
  def __init__(self, txt, flag):
    if flag == 1:
      self.txt = ipywidgets.HTML(value='<p style="text-align: left;style=font-family:Arial"><span style="color: #339966;">' + txt + '</span></p>')
    if flag == 2:
      self.txt = ipywidgets.HTML(value='<p style="text-align: left;style=font-family:Arial"><span style="color: #ff0000;">' + txt + '</span></p>')

# Função para a conexão com o GDrive
def drive_connect():
  drive.mount('/content/gdrive', force_remount=True)
  drive_connect.root_dir = "/content/gdrive/MyDrive"
  return True

# Atributos do Dataset
os.environ["DATASET_DIRECTORY"] = ""
dataset_total_images = 0
dataset_dict_labels = {}

# Layouts e Styles
l_txt = ipywidgets.Layout(width='66%')
l_btn = ipywidgets.Layout(width='33%')
l_vbox = ipywidgets.Layout(width='720px')
l_hbox = ipywidgets.Layout(width='90%')
l_req = ipywidgets.Layout(width='95%', height='90px')
l_hidden_red = ipywidgets.Layout(width='90%', visibility = 'hidden')
l_hidden_tra = ipywidgets.Layout(width='90%', visibility = 'hidden')
l_btn_hbox = ipywidgets.Layout(flex_flow='column',align_items='center')
s_desc_wid = {'description_width':'25%'}

# Modelos do yolov5
models = ['yolov5n', 'yolov5s', 'yolov5m', 'yolov5l', 'yolov5x', 'yolov5n6', 'yolov5s6', 'yolov5m6', 'yolov5l6', 'yolov5x6']

# Lista de tipos para exportar
export_list = ['tflite', 'torchscript', 'pb', 'onnx', 'pt']

def total_by_category():
  folders = Path(os.environ["DATASET_DIRECTORY"]).ls()
  totalImages = 0
  dictCategory = {}
  fileDataYaml = open(os.environ["DATASET_DIRECTORY"] + "/data.yaml")
  try:
    data = fileDataYaml.read()
    classes = re.search(r'\[( ?\'[A-z ]+\',?)+\]', data).group().replace("[", "").replace("]", "").replace("'", "").split(", ")
    for _class in classes:
      dictCategory[_class] = 0
    for folder in folders:
      if os.path.isfile(folder):
        continue
      if Path(folder/"labels").is_dir() and Path(folder/"images").is_dir():
        for filePath in Path(folder/"labels").iterdir():
          fileLabel = open(filePath)
          for line in fileLabel.readlines():
            numberClass = line.split(" ")[0]
            dictCategory[classes[int(numberClass)]] += 1
      else:
        raise Exception("Erro no Conjunto dos" + folder)
  except:
    print("Erro na organização do Dataset")

  return totalImages, dictCategory

def renderCategories(totalImages, categoryDict):
  chart_labels = [] 
  chart_quantities = []
  for i in categoryDict:
    chart_labels.append(i)
    chart_quantities.append(categoryDict[i])

  x = np.arange(len(chart_labels))  # the label locations
  width = 0.5  # the width of the bars
  
  fig, ax = plt.subplots()
  rects = ax.bar(x, chart_quantities, width)
  
  # Add some text for labels, title and custom x-axis tick labels, etc.
  ax.set_ylabel('Quantidade')
  ax.set_title('Quantidade por categoria')
  ax.set_xticks(x)
  ax.set_xticklabels(chart_labels)
  ax.set_frame_on(False)
  #ax.bar_label(rects, padding=3) so roda a partir de matplotlib 3.2.4 (fastai importa 3.2.2)
  
  #assim para colocar os labels em cima de cada coluna necessita:
  def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
      height = rect.get_height()
      ax.annotate('{}'.format(height),
                  xy=(rect.get_x() + rect.get_width() / 2, height),
                  xytext=(0, 3),  # 3 points vertical offset
                  textcoords="offset points",
                  ha='center', va='bottom')

  autolabel(rects)
  opt = ipywidgets.Output()
  with opt:
    plt.rcParams["figure.figsize"] = (25, 15)
    plt.rcParams.update({'font.size': 22})
    plt.show()
  
  i = [opt]
  display(ipywidgets.VBox(i, layout=l_vbox))

# Função que renderiza a janela de Analise De Requisitos
def renderAnaliseDeRequisitos(tab):
  with tab.output_to(0, select=True):  
    grid0 = colabwidigets.Grid(1, 1)
    with grid0.output_to(0,0):
      lbl_tarefa = ipywidgets.Label(value = "Tarefa:")
      txta_tarefa = ipywidgets.Textarea(value='',placeholder="Descreva a tarefa \nex. Detecção de objetos aplicadoa a 10 objetos de dentro de casa.",disabled=False,layout=l_req)
      
      lbl_tipotarefa = ipywidgets.Label(value = "Tipo da tarefa:")
      txta_tipotarefa = ipywidgets.Textarea(value='',placeholder='Descreva o tipo da tarefa \nex. Detecção de Objetos',disabled=False,layout=l_req)

      lbl_cat = ipywidgets.Label(value = "Categorias:")
      txta_cat1 = ipywidgets.Textarea(value='',placeholder='Descrição das categorias \nex. 10 categorias de objetos encontrados dentro de casa, que podem interferir no dia a dia de uma pessoa com deficiência visual',disabled=False,layout=l_req)

      lbl_fontededados = ipywidgets.Label(value = "Fonte de dados:")
      txta_fontededados = ipywidgets.Textarea(value='',placeholder='Descreva a fonte de dados \nex. Conjunto de dados disponibilizado ou importado pelo usuário',disabled=False,layout=l_req)

      lbl_qtddedados = ipywidgets.Label(value = "Quantidade de dados:")
      txta_qtddedados = ipywidgets.Textarea(value='',placeholder='Descreva a quantidade de dados \nex. No mínimo 30 exemplares para cada categoria',disabled=False,layout=l_req)
      
      lbl_pdr_img = ipywidgets.Label(value = "Padronização das imagens:")
      txta_pdr_img1 = ipywidgets.Textarea(value='',placeholder='Descrição do formato das imagens \nex. Formato: .jpeg, Tamanho: 416x416 pixels',disabled=False,layout=l_req)

      lbl_rotulacao = ipywidgets.Label(value = "Rotulação dos dados:")
      txta_rotulacao = ipywidgets.Textarea(value='',placeholder='Descreva a rotulação dos dados \nex. Por estudantes do ensino superior (coleta própria)',disabled=False,layout=l_req)

      lbl_desempenho = ipywidgets.Label(value = "Desempenho:")
      txta_desempenho = ipywidgets.Textarea(value='',placeholder='Descreva o desempenho desejado \nex. mAP (Precisão média): No mínimo 75%, F1 score: No mínimo 0.75',disabled=False,layout=l_req)      

      itens_req = ipywidgets.VBox([ipywidgets.VBox([lbl_tarefa, txta_tarefa], layout=l_hbox),
                                    ipywidgets.VBox([lbl_tipotarefa, txta_tipotarefa], layout=l_hbox),
                                    ipywidgets.VBox([lbl_cat, txta_cat1], layout=l_hbox),
                                    ipywidgets.VBox([lbl_fontededados, txta_fontededados], layout=l_hbox),
                                    ipywidgets.VBox([lbl_qtddedados, txta_qtddedados], layout=l_hbox),
                                    ipywidgets.VBox([lbl_pdr_img, txta_pdr_img1], layout=l_hbox),
                                    ipywidgets.VBox([lbl_rotulacao, txta_rotulacao], layout=l_hbox),
                                    ipywidgets.VBox([lbl_desempenho, txta_desempenho], layout=l_hbox)],layout=l_vbox)

      display(itens_req)

# Função que renderiza a janela para preparação dos dados
def renderPreparacaoDosDados(tab):
  with tab.output_to(1, select=False):  
    grid1 = colabwidigets.Grid(4,1)
    with grid1.output_to(0,0):
      lbl_path = m_Header('Definição do caminho do Dataset')
      input_path = ipywidgets.Text(placeholder='Insira o caminho do pasta do conjunto de dados no google drive',value='',layout=l_txt)
      btn_path = ipywidgets.Button(description='Carregar o Caminho', layout=l_btn)
      hbox_path = ipywidgets.HBox([input_path, btn_path], layout=l_hbox)
      vbox_path = ipywidgets.VBox([lbl_path.header, hbox_path], layout=l_vbox)
      display(vbox_path)

      def on_btn_path_clicked(a):
        with grid1.output_to(1,0):
          grid1.clear_cell()
          os.environ["DATASET_DIRECTORY"] = drive_connect.root_dir + input_path.value
          if Path(os.environ["DATASET_DIRECTORY"]).is_dir():
            msg_ret = m_Ret('Path: ' + os.environ["DATASET_DIRECTORY"] + '/ definido com sucesso', 1)
          else:
            msg_ret = m_Ret('Path: ' + os.environ["DATASET_DIRECTORY"] + '/ não encontrado', 2)
          display(msg_ret.txt)

      btn_path.on_click(on_btn_path_clicked)

  # Atividade - Visarlizar infos do dataset
  with grid1.output_to(2, 0):
    lbl_dataset_info = ipywidgets.Label(value = "Visualizar informações do conjunto de dados")

    btn_view_total_by_category = ipywidgets.Button(description='Ver objetos por categoria', layout=l_btn)
    hbox_dataset_info = [lbl_dataset_info, ipywidgets.Box([btn_view_total_by_category], layout=l_btn_hbox)]
    vbox_dataset_info = ipywidgets.VBox(hbox_dataset_info, layout=l_vbox)
    display(vbox_dataset_info)
    
    def on_btn_view_total_by_category(_):
      with grid1.output_to(3, 0):
        dataset_total_images, dataset_dict_labels = total_by_category()
        renderCategories(dataset_total_images, dataset_dict_labels)

    btn_view_total_by_category.on_click(on_btn_view_total_by_category)

# Função que renderiza a janela para o transfer learning
def renderTransferLearning(tab):
  with tab.output_to(2, select=False): #):  
    grid2 = colabwidigets.Grid(2,1)
    with grid2.output_to(0,0):
      lbl_trainning = m_Header('Treinamento do modelo')

      model = ipywidgets.Dropdown(
          options=models,
          value=models[0],
          description='Modelo:',
          disabled=False,
      )
  
      slider_batches = ipywidgets.IntSlider(min = 1,
                                              max = 100,
                                              step = 1,
                                              value = 16,
                                              description_tooltip='numero de lotes',
                                              description='Batches',
                                              layout=l_hbox,
                                              style=s_desc_wid)
      slider_image_size = ipywidgets.IntSlider(min = 64,
                                              max = 1280,
                                              step = 64,
                                              value = 640,
                                              description_tooltip='tamanho das imagens',
                                              description='ImageSize',
                                              layout=l_hbox,
                                              style=s_desc_wid)
      slider_epochs = ipywidgets.IntSlider(min = 1,
                                              max = 200,
                                              step = 1,
                                              value = 50,
                                              description_tooltip='número de épocas',
                                              description='Epochs',
                                              layout=l_hbox,
                                              style=s_desc_wid)
     

      btn_train_model = ipywidgets.Button(description='Treinar Modelo',tooltip='',layout=l_btn)
      hbox_trainning = [lbl_trainning.header, model, slider_batches, slider_image_size, slider_epochs, ipywidgets.Box([btn_train_model], layout=l_btn_hbox)]
      vbox_trainning = ipywidgets.VBox(hbox_trainning, layout=l_vbox)
      display(vbox_trainning)

      def on_btn_train_model_clicked(b):
        with grid2.output_to(1,0):
          os.environ["BATCHES"] = str(slider_batches.value)
          os.environ["IMAGE_SIZE"] = str(slider_image_size.value)
          os.environ["EPOCHS"] = str(slider_epochs.value)
          os.environ["MODEL"] = str(model.value)
          createConfigFiles()
          if Path("./yolov5/runs/train").is_dir():
            !rm -rf yolov5/runs/train
          !python yolov5/train.py --img ${IMAGE_SIZE} --batch ${BATCHES} --epochs ${EPOCHS} --data ${DATASET_DIRECTORY}/data.yaml --weights yolov5/${MODEL}.pt
      
      btn_train_model.on_click(on_btn_train_model_clicked)

# Render Avaliação
def renderAvaliacao(tab):
  with tab.output_to(3, select=False):  
    grid6 = colabwidigets.Grid(6,1)

    with grid6.output_to(0,0):
      lbl_results = ipywidgets.Label(value='Avaliação Geral')
      btn_results = ipywidgets.Button(description='Mostrar', layout=l_btn, style=s_desc_wid)

      box_predict = ipywidgets.VBox([lbl_results, ipywidgets.HBox([btn_results], layout=l_btn_hbox)], layout=l_vbox)
      display(box_predict)

      def on_btn_results(b):
        with grid6.output_to(1,0):
          results = open("/content/yolov5/runs/train/exp/results.png", "rb")
          results_image = results.read()
          results_image_widget = ipywidgets.Image(
              value=results_image,
              format='png',
              width=720,
              height=720,
          )
          display(results_image_widget)

      btn_results.on_click(on_btn_results)

    with grid6.output_to(2,0):
      lbl_f1 = ipywidgets.Label(value='Curva do F1 Score')
      btn_f1 = ipywidgets.Button(description='Mostrar', layout=l_btn, style=s_desc_wid)

      box_predict = ipywidgets.VBox([lbl_f1, ipywidgets.HBox([btn_f1], layout=l_btn_hbox)], layout=l_vbox)
      display(box_predict)

      def on_btn_f1(b):
        with grid6.output_to(3,0):
          f1_curve = open("/content/yolov5/runs/train/exp/F1_curve.png", "rb")
          f1_curve_image = f1_curve.read()
          f1_curve_image_widget = ipywidgets.Image(
              value=f1_curve_image,
              format='png',
              width=720,
              height=720,
          )
          display(f1_curve_image_widget)

      btn_f1.on_click(on_btn_f1)

    with grid6.output_to(4,0):
      lbl_matrix = ipywidgets.Label(value='Matriz de Confusão')
      btn_matrix = ipywidgets.Button(description='Mostrar', layout=l_btn, style=s_desc_wid)

      box_predict = ipywidgets.VBox([lbl_matrix, ipywidgets.HBox([btn_matrix], layout=l_btn_hbox)], layout=l_vbox)
      display(box_predict)

      def on_btn_confusion_matrix(b):
        with grid6.output_to(5,0):
          confusion_matrix = open("/content/yolov5/runs/train/exp/confusion_matrix.png", "rb")
          confusion_matrix_image = confusion_matrix.read()
          confusion_matrix_image_widget = ipywidgets.Image(
              value=confusion_matrix_image,
              format='png',
              width=720,
              height=720,
          )
          display(confusion_matrix_image_widget)

      btn_matrix.on_click(on_btn_confusion_matrix)

# Render Predição
def renderPredicao(tab):
  with tab.output_to(4, select=False):  
    grid6 = colabwidigets.Grid(5,1)
    slider_confidence = ipywidgets.IntSlider(min = 1,
                                            max = 100,
                                            step = 1,
                                            value = 10,
                                            description_tooltip='Taxa de Confiança',
                                            description='Confiança',
                                            layout=l_hbox,
                                            style=s_desc_wid)

    with grid6.output_to(0,0):
      lbl_confidence = m_Header('Taxa de Confiança')

      box_confidence = ipywidgets.VBox([lbl_confidence.header, slider_confidence], layout=l_vbox)
      display(box_confidence)

    with grid6.output_to(1,0):
      lbl_prediction = ipywidgets.Label(value="Predição do seu Dataset")

      btn_pred = ipywidgets.Button(description='Predição', layout=l_btn, style=s_desc_wid)
      box_predict = ipywidgets.VBox([lbl_prediction, ipywidgets.HBox([btn_pred], layout=l_btn_hbox)], layout=l_vbox)
      display(box_predict)

      def on_btn_pred(b):
        with grid6.output_to(2,0):
          if Path("./yolov5/runs/detect").is_dir():
            !rm -rf yolov5/runs/detect
    
          os.environ["CONFIDENCE"] = str(slider_confidence.value/100)
          !python yolov5/detect.py --weights yolov5/runs/train/exp/weights/best.pt --img ${IMAGE_SIZE} --conf ${CONFIDENCE} --source ${DATASET_DIRECTORY}/test/images
          for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'):
            display(Image(filename=imageName))
            print("\n")

      btn_pred.on_click(on_btn_pred)

    with grid6.output_to(3,0):
      lbl_prediction_upload = ipywidgets.Label(value="Predição pelo upload")

      btn_pred = ipywidgets.Button(description='Predição', layout=l_btn, style=s_desc_wid)
      box_predict = ipywidgets.VBox([lbl_prediction_upload, ipywidgets.HBox([btn_pred], layout=l_btn_hbox)], layout=l_vbox)
      display(box_predict)

      def on_btn_pred(b):
        with grid6.output_to(4,0):
          if Path("./yolov5/runs/detect").is_dir():
            !rm -rf yolov5/runs/detect
    
          os.environ["CONFIDENCE"] = str(slider_confidence.value/100)
          !python yolov5/detect.py --weights yolov5/runs/train/exp/weights/best.pt --img ${IMAGE_SIZE} --conf ${CONFIDENCE} --source ${DATASET_DIRECTORY}/test/images
          for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'):
            display(Image(filename=imageName))
            print("\n")

      btn_pred.on_click(on_btn_pred)

# Render Exportação
def renderExportacao(tab):
  with tab.output_to(5, select=False):  
    grid7 = colabwidigets.Grid(3,1)
    with grid7.output_to(0,0):
      lbl_export = m_Header('Exportação do modelo')
      lbl_model = ipywidgets.Label(value = "Formato:")
      lbl_path = ipywidgets.Label(value = "Caminho:")

      model_export = ipywidgets.Dropdown(
          options=export_list,
          value=export_list[0],
          disabled=False,
      )
      export_path = ipywidgets.Text(placeholder='Insira um caminho de um diretório válido no drive',value='',layout=l_txt)
      btn_export = ipywidgets.Button(description='Exportar', layout=l_btn)
      vbox_dropdown = ipywidgets.VBox([lbl_model, model_export], layout=l_vbox)

      btn_path = ipywidgets.Button(description='Carregar Caminho', layout=l_btn)
      hbox_export = ipywidgets.HBox([export_path, btn_path], layout=l_hbox)
      hbox_btn_export = ipywidgets.HBox([btn_export], layout=l_btn_hbox)
      vbox_export = ipywidgets.VBox([lbl_export.header, vbox_dropdown, lbl_path, hbox_export, hbox_btn_export], layout=l_vbox)

      display(vbox_export)

      def on_btn_path_clicked(a):
        with grid7.output_to(1,0):
          grid7.clear_cell()
          os.environ["EXPORT_DIRECTORY"] = drive_connect.root_dir + export_path.value
          if Path(os.environ["EXPORT_DIRECTORY"]).is_dir():
            msg_ret = m_Ret('Path: ' + os.environ["EXPORT_DIRECTORY"] + '/ definido com sucesso', 1)
          else:
            msg_ret = m_Ret('Path: ' + os.environ["EXPORT_DIRECTORY"] + '/ não encontrado', 2)
          display(msg_ret.txt)
      btn_path.on_click(on_btn_path_clicked)

      def on_btn_export_clicked(b):
        with grid7.output_to(2,0):
          grid7.clear_cell()
          os.environ["MODEL_EXPORT"] = str(model_export.value)
          if Path(os.environ["EXPORT_DIRECTORY"]).is_dir():
            if os.environ["MODEL_EXPORT"] != 'pt':
              !python yolov5/export.py --weights yolov5/runs/train/exp/weights/best.pt --img ${IMAGE_SIZE} --include ${MODEL_EXPORT}
            if os.environ["MODEL_EXPORT"] == 'tflite':
              !cp yolov5/runs/train/exp/weights/best-fp16.tflite ${EXPORT_DIRECTORY}
            else:            
              !cp yolov5/runs/train/exp/weights/best.${MODEL_EXPORT} ${EXPORT_DIRECTORY}  
            msg_ret = m_Ret('Exportado para: \'' + os.environ["EXPORT_DIRECTORY"] + ' com sucesso', 1)
            display(msg_ret.txt)
          else:
            msg_ret = m_Ret('Diretorio: \'' + os.environ["EXPORT_DIRECTORY"] + ' não encontrado', 2)
            display(msg_ret.txt)

      btn_export.on_click(on_btn_export_clicked)

def createConfigFiles():
  with open(os.environ["DATASET_DIRECTORY"] + "/data.yaml", 'r+') as f:
    text = f.read()
    text = re.sub(r'train: ([A-z0-9/.-]+)', "train: " + os.environ["DATASET_DIRECTORY"] + "/train/images", text)
    text = re.sub(r'val: ([A-z0-9/.-]+)', "val: " + os.environ["DATASET_DIRECTORY"] + "/valid/images", text)
    f.seek(0)
    f.write(text)
    f.truncate()  

def chargeYolov5():
  #clone YOLOv5 and 
  !git clone https://github.com/ultralytics/yolov5 | tee -a >> log.txt  # clone repo
  %cd yolov5
  %pip install -qr requirements.txt
  %pip install -q roboflow
  %cd ..

# Função para ser executada, encapsula tudo
def renderUI():
  # Carrega os arquivos do Darknet
  chargeYolov5()
  # Conecta ao Google Drive
  connected = drive_connect()
  clear_output(wait=True)
  if connected:
    # Criação do painel que vai conter as abas
    tab = colabwidigets.TabBar(['Análise de requisitos', 'Preparação de dados', 'Transfer Learning', 'Avaliação', 'Predição', 'Exportação'])
    # Renderização da Analise de Requisitos  
    renderAnaliseDeRequisitos(tab)
    # Renderização da Preparação dos Dados
    renderPreparacaoDosDados(tab)
    # Renderização para o Transfer Learning
    renderTransferLearning(tab)
    # Renderização para a avaliação
    renderAvaliacao(tab)
    # Renderização para a predição
    renderPredicao(tab)
    # Renderização para a Exportação
    renderExportacao(tab)


In [ ]:
renderUI()